In [5]:
"""
TSP for the EL bars. 
A fair chunk of this code is shamelessly lifted from Randy Olsen
Thank you! 

https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb
"""

import googlemaps
from datetime import datetime

# load API key from file not included in repo
apikey = None
with open("./key.txt", 'r') as keyfile:
    apikey = keyfile.readline().strip()

#set up API client
gmaps = googlemaps.Client(key=apikey)

#walking to the bars
tmode = "walking"

#read in json of bars to get addresses
import json 

jsonData=None 
with open("bars.json","r") as iFile: 
    jsonData = json.load(iFile)

#Thank you Randy Olsen! 
from itertools import combinations

#Save this data in a space seperated file ala 830 Hacker Rannk 
outfile=open("distances.ssv","w") 
outfile.write("bar1, bar2, seconds, meters\n")
for (point1, point2) in combinations(jsonData,2): 
    address1=point1["address"]  
    bar1=point1["bar"]
    
    address2=point2["address"]
    bar2=point2["bar"]
    
    route = gmaps.distance_matrix(origins=[address1],
                                  destinations=[address2],
                                  mode=tmode,
                                  language="English",
                                  units="metric"
                                 )
    
    secondsApart=route["rows"][0]["elements"][0]["duration"]["value"]
    metersApart=route["rows"][0]["elements"][0]["distance"]["value"]
    
    
    #write all of the things!
    
    outfile.write("{} {} {} {}\n".format(bar1, bar2, secondsApart, metersApart))
    print bar1, bar2, secondsApart, metersApart 
    
    print route 
    break
outfile.close()


ricks pts 25 33
{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'1 min', u'value': 25}, u'distance': {u'text': u'33 m', u'value': 33}, u'status': u'OK'}]}], u'origin_addresses': [u'224 Abbot Rd, East Lansing, MI 48823, USA'], u'destination_addresses': [u'210 Abbot Rd, East Lansing, MI 48823, USA']}
